**Kütüphaneleri Ekleyelim**

In [0]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

**Sitemizden Request Alalım**

In [2]:
url="http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1"
cevap=get(url)
print(cevap.text[:200])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatibl


**BeautifulSoup ile HTML içeriğini ayrıştıralım**

In [0]:
html_soup=BeautifulSoup(cevap.text,"html.parser")

**Sitenin HTML yapısına göz attıktan sonra verileri ayrıştırmaya başlayalım**

In [5]:
film_icerikleri=html_soup.find_all("div",class_="lister-item mode-advanced")
print(len(film_icerikleri))

50


**film_icerikleri değişkenimizin içerisine  bir liste gibi erişim sağlayabiliyoruz.**

In [0]:
#film_icerikleri[0]

**İlk listedeki filmin adına ulaşalım.**

In [13]:
ilk_film=film_icerikleri[0]
ilk_isim=ilk_film.h3.a.text
ilk_isim

'Logan'

**İlk filmin çıkış yılına ulaşalım.**

In [14]:
ilk_yil=ilk_film.h3.find("span",class_="lister-item-year text-muted unbold")
ilk_yil.text

'(2017)'

**İlk filmin IMDB puanına ulaşalım.**

In [15]:
ilk_imdb=float(ilk_film.strong.text)
ilk_imdb

8.1

**İlk filmin METASCORE puanına ulaşalım.**

In [16]:
ilk_mscore=ilk_film.find("span",class_="metascore favorable")
ilk_mscore=int(ilk_mscore.text)
ilk_mscore

77

**İlk filmin OY sayısına ulaşalım.**

In [17]:
ilk_oy=ilk_film.find("span",attrs={'name':'nv'})
print(ilk_oy)
ilk_oy=int(ilk_oy["data-value"])
ilk_oy

<span data-value="559637" name="nv">559,637</span>


559637

**Bu noktada sayfaya baktığımızda METASCORE puanı olmayan değerler görüyoruz. Örneğin 23. Sıradakı  13 Reasons Why filmi**

In [19]:
kontrol=film_icerikleri[22]
kontrol=kontrol.find("div",class_="ratings-metascore")
print(kontrol)
type(kontrol)

None


NoneType

**Değişken tipi NoneType olarak dönüyor ve text'e çevirirken hata alıyoruz. Bu nedenle kontrol ediyoruz.**

Hata:

In [20]:
kontrol=kontrol.find("div",class_="ratings-metascore").text

AttributeError: ignored

**Şimdi bu ayrı ayrı yazdığımız kodları sayfadaki bütün filmlerin değerlerini almak için birleştirelim.**

**Not:** Metascore puanlarının HTML kodu içerisinde **metascore favorable** ve **metascore mixed** olarak 2 şekilde tanımlandığının fark edilmesiyle bu kontrolü de yapıyoruz.

In [24]:
isimler=[]
yıllar=[]
imdbs=[]
metascore=[]
oylar=[]
for icer in film_icerikleri:
  if icer.find("div",class_="ratings-metascore") is not None:
    #İSİM
    isim=icer.h3.a.text
    isimler.append(isim)
    #YIL
    yıl=icer.h3.find('span', class_ = 'lister-item-year').text
    yıllar.append(yıl)
    #IMDB
    imdb=float(icer.strong.text)
    imdbs.append(imdb)
    #METASCORE
    if icer.find("span",class_="metascore favorable") is not None:
      score=icer.find("span",class_="metascore favorable").text
    elif icer.find("span",class_="metascore mixed") is not None: 
      score=icer.find("span",class_="metascore mixed").text    
    metascore.append(int(score))
    #OY
    oy=int(icer.find("span",attrs={'name':'nv'})["data-value"])
    oylar.append(oy)
#Elde ettiğimiz verileri DataFrame olarak kaydediyoruz.
df=pd.DataFrame({"film":isimler,
                "yıl":yıllar,
                "imdb":imdbs,
                "metascore":metascore,
                "oylar":oylar})
df.head(6)

,film,yıl,imdb,metascore,oylar
0,Logan,(2017),8.1,77,559637
1,Wonder Woman,(2017),7.5,76,487150
2,Guardians of the Galaxy Vol. 2,(2017),7.7,67,482283
3,Thor: Ragnarok,(2017),7.9,74,482079
4,Dunkirk,(2017),7.9,94,466124
5,Star Wars: Episode VIII - The Last Jedi,(2017),7.1,85,462506


**Kaynak:** https://www.dataquest.io/blog/web-scraping-beautifulsoup/